In [11]:
import numpy as np
import matplotlib.pyplot as plt
from MJOLNIR.TasUBlibDEG import calTwoTheta, calculateBMatrix, calcCell
from MJOLNIR.Geometry.Instrument import prediction, converterToA3A4, converterToQxQy
from MJOLNIR.Data import Sample
import MJOLNIR

class PredictionToolManager:
    def __init__(self):
        self.predictionAX = None
        self.sample = None
        self.Efs = None
        self.A4Instrument = None
        self.loadEnergies()

    def updateSample(self, cell, alignment1, alignment2):
        self.sample = Sample.Sample(*cell, projectionVector1=alignment1, projectionVector2=alignment2)
        self.calcualteHKLtoA3A4()

    def getAlignment(self, alignment):
        H, K, L, A3, A4, Ei, Ef = alignment
        return [H, K, L, A3, A4, 0.0, 0.0, Ei, Ef]  # H, K, L, A3, A4, phi, chi, Ei, Ef

    def setAlignment(self, R, alignment):
        H, K, L, A3, A4, Ei, Ef = R
        alignment[0] = H
        alignment[1] = K
        alignment[2] = L
        alignment[3] = A3
        alignment[4] = A4
        alignment[5] = Ei
        alignment[6] = Ef

    def calculateA4(self, alignment):
        cell = self.getCell()
        r = self.getAlignment(alignment=alignment)
        Cell = calcCell(cell)
        B = calculateBMatrix(Cell)
        qe = np.concatenate([r[:3], r[-2:]])
        A4 = calTwoTheta(B=B, qe=qe, ss=-1)
        return A4

    def loadEnergies(self, instrument='CAMEA'):
        if instrument == 'CAMEA':
            calibPath = MJOLNIR.__CAMEANormalizationBinning1__
            calib = np.loadtxt(calibPath, delimiter=',', skiprows=3)
            self.A4Instrument = calib[:, -1].reshape(104, 8)
            self.EfInstrument = calib[:, 4].reshape(104, 8)
            self.EfInstrument[np.isclose(self.EfInstrument, 0.0)] = np.nan
        else:
            raise NotImplementedError('Currently only CAMEA is supported.')
        self.Efs = np.nanmean(self.EfInstrument, axis=0)[::-1]

    def validateR(self, r):
        H, K, L, _, A4, _, _, Ei, Ef = r
        if np.isclose(np.linalg.norm([H, K, L]), 0.0):
            return False
        if np.isclose(A4, 0.0):
            return False
        if np.any([np.isclose(E, 0.0) for E in [Ei, Ef]]):
            return False
        return True

    def generatePrediction(self, scan, alignment1, alignment2, cell, instrument='CAMEA'):
        A3Start, A3Stop, A3Steps, Ei, A4, points, Monitor = scan
        r1 = np.array(self.getAlignment(alignment=alignment1))
        r2 = np.array(self.getAlignment(alignment=alignment2))
        sample = Sample.calculateSample(cell=cell, HKL1=r1[:3], HKL2=r2[:3], A3R1=r1[3], A3R2=r2[3], Ei=r1[-2], Ef=r1[-1])
        ax = prediction(A3Start=A3Start, A3Stop=A3Stop, A3Steps=A3Steps, A4Positions=A4, Ei=Ei, sample=sample, points=points, instrument=instrument)
        self.predictionAx = ax

        for a in ax[:-1]:
            a.set_ylim(*ax[-1].get_ylim())
            a.set_xlim(*ax[-1].get_xlim())

        plt.show()

    def getScan(self, A3Start, A3Stop, A3Steps, Ei, A4String, Monitor):
        A4 = self.formatA4String(A4String)
        points = True
        return A3Start, A3Stop, A3Steps, Ei, A4, points, Monitor

    def formatA4String(self, A4String):
        return [float(x) for x in A4String.split(',') if x]

    def getCell(self, a, b, c, alpha, beta, gamma):
        return a, b, c, alpha, beta, gamma

    def calcualteHKLtoA3A4(self, Ei, Ef, H, K, L):
        """
        Calculate A3 and A4 angles from HKL values and energies.

        Parameters:
        - Ei: Incident energy
        - Ef: Final energy
        - H, K, L: Miller indices

        Returns:
        - A3, A4: Calculated angles
        """
        UB = self.sample.UB  # Get the UB matrix from the sample
        Qx, Qy, _ = np.dot(UB, [H, K, L])  # Transform HKL to Q-space
        A3, A4 = converterToA3A4(Qx, Qy, Ei, Ef, A3Off=0.0, A4Sign=np.sign(self.sample.plane_vector1[4]))  # Convert to angles
        return A3, A4


    def calcualteA3A4toHKL(self, A3, A4, Ei, Ef):
        Qx, Qy = converterToQxQy(A3, A4, Ei, Ef)
        H, K, L = self.sample.calculateQxQyToHKL(Qx, Qy)
        return H, K, L

    def getCalculation(self, Ei, EfIndex, H, K, L, A3, A4):
        if EfIndex == len(self.Efs):
            Ef = Ei
        else:
            Ef = self.Efs[EfIndex]
        return Ei, Ef, H, K, L, A3, A4


In [12]:
import numpy as np

# Assuming the refactored PredictionToolManager class is already defined and imported

def run_prediction_tool():
    # Initialize the prediction tool manager
    manager = PredictionToolManager()
    
    # Define a sample unit cell (a, b, c, alpha, beta, gamma)
    cell = (5.43, 5.43, 5.43, 90.0, 90.0, 90.0)  # Example cubic cell, typical for silicon

    # Define two alignments with H, K, L, A3, A4, phi, chi, Ei, Ef values
    alignment1 = np.array([1, 0, 0, 10.0, 20.0, 0.0, 0.0, 14.7, 14.7])  # Alignment vector 1 as numpy array
    alignment2 = np.array([0, 1, 0, 15.0, 25.0, 0.0, 0.0, 14.7, 14.7])  # Alignment vector 2 as numpy array

    # Update the sample configuration in the manager
    manager.updateSample(cell, alignment1, alignment2)

    # Perform HKL to A3, A4 calculation using the refactored method
    Ei = 14.7
    Ef = 14.7  # Assuming Ei = Ef for this example
    H, K, L = 1, 0, 0  # Example HKL values
    A3, A4 = manager.calcualteHKLtoA3A4(Ei, Ef, H, K, L)
    print(f"Calculated A3: {A3}, A4: {A4}")

    # Define scan parameters: A3 start/stop, number of steps, incident energy, A4 positions, and monitor value
    A3Start = 0.0  # Starting angle for A3
    A3Stop = 60.0  # Ending angle for A3
    A3Steps = 10   # Number of steps in the scan
    Ei = 14.7      # Incident energy in meV
    A4String = "20.0,30.0,40.0"  # Example A4 positions
    Monitor = 1000  # Monitor counts

    # Get the formatted scan parameters tuple
    scan = manager.getScan(A3Start, A3Stop, A3Steps, Ei, A4String, Monitor)
    
    # Generate the prediction using the provided scan parameters and sample configuration
    manager.generatePrediction(scan, alignment1, alignment2, cell, instrument='CAMEA')

    print("Prediction generation complete.")

if __name__ == "__main__":
    run_prediction_tool()


TypeError: PredictionToolManager.calcualteHKLtoA3A4() missing 5 required positional arguments: 'Ei', 'Ef', 'H', 'K', and 'L'